ImageNet example

* copy the whole directory 'imagenet_install' available at https://drive.google.com/drive/folders/1DI0LW0cSeeGM_NFVbdMOtp3JDQk8JCUu?usp=sharing
 
> * upload it to drive/My Drive/public/. (make sure you have a directory 'drive/My Drive/public/imagenet_install')

* Move the public/images folder to 'drive/My Drive/public/data/images/.'

**Outline**
* Image Classification with Pre-trained Model

 * Mount Google Drive

 * Download and load pre-trained model

 * Image classification with pre-trained model

 In pre-trained model experiment, data and results will be stored in,
 1. Imagenet Data: './drive/My Drive/public/data/images'
 2. Trained Model: './drive/My Drive/public/results/'

\\

* Transfer Learning with pre-trained Model

 * Load pre-trained model

 * Configure optimizer: only training the last layer

 * Load dataset

 * Training

 * Testing

 * Image 

 In fine-tuning experiment, toy dataset and results are stored in,
 1. './drive/My Drive/public/data/images'
 2. './drive/My Drive/public/results/'
 

# New Section

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


configuration to download a pretrained model 

In [6]:
!pip install pretrainedmodels
!python ./drive/'My Drive'/public/imagenet_install/setup.py install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
running install
running bdist_egg
running egg_info
creating pretrainedmodels.egg-info
writing pretrainedmodels.egg-info/PKG-INFO
writing dependency_links to pretrainedmodels.egg-info/dependency_links.txt
writing requirements to pretrainedmodels.egg-info/requires.txt
writing top-level names to pretrainedmodels.egg-info/top_level.txt
writing manifest file 'pretrainedmodels.egg-info/SOURCES.txt'
reading manifest file 'pretrainedmodels.egg-info/SOURCES.txt'
writing manifest file 'pretrainedmodels.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib

creating build
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/EGG-INFO
copying pretrainedmodels.egg-info/PKG-INFO -> build/bdist.linux-x86_64/egg/EGG-INFO
copying pretrainedmodels.egg-info/SOURCES.txt -> build/bdist.linux-x86_64/egg/EGG-INFO

download a pretrained model and prepare a transformation function

In [10]:
import pretrainedmodels
import pretrainedmodels.utils as utils
import torch
import os
import warnings
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
warnings.filterwarnings("ignore")


In [11]:
# load the model if you have downloaded and saved a model. 
model_name = 'nasnetalarge' # could be fbresnet152 or inceptionresnetv2
model_file = 'drive/My Drive/public/results/imagenet_nasnetalarge.pth'
if os.path.exists(model_file):
  model = torch.load(model_file)
else:
  model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
  if not os.path.exists('drive/My Drive/public/results'):
      os.mkdir('drive/My Drive/public/results')    
  torch.save(model, model_file)

model.eval()

import pretrainedmodels.utils as utils
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

load_img = utils.LoadImage()
transform_img = utils.TransformImage(model) #input_size = (331, 331) 

imagenet class names 

In [12]:
import csv
name_file = 'drive/My Drive/public/imagenet_install/class_names.csv'

imagenet_class  = {} 
file_in = csv.reader(open(name_file))
for row in file_in:
  imagenet_class[int(row[0])] = row[1]

In [13]:
print(imagenet_class)

{0: 'tench, Tinca tinca', 1: 'goldfish, Carassius auratus', 2: 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias', 3: 'tiger shark, Galeocerdo cuvieri', 4: 'hammerhead, hammerhead shark', 5: 'electric ray, crampfish, numbfish, torpedo', 6: 'stingray', 7: 'cock', 8: 'hen', 9: 'ostrich, Struthio camelus', 10: 'brambling, Fringilla montifringilla', 11: 'goldfinch, Carduelis carduelis', 12: 'house finch, linnet, Carpodacus mexicanus', 13: 'junco, snowbird', 14: 'indigo bunting, indigo finch, indigo bird, Passerina cyanea', 15: 'robin, American robin, Turdus migratorius', 16: 'bulbul', 17: 'jay', 18: 'magpie', 19: 'chickadee', 20: 'water ouzel, dipper', 21: 'kite', 22: 'bald eagle, American eagle, Haliaeetus leucocephalus', 23: 'vulture', 24: 'great grey owl, great gray owl, Strix nebulosa', 25: 'European fire salamander, Salamandra salamandra', 26: 'common newt, Triturus vulgaris', 27: 'eft', 28: 'spotted salamander, Ambystoma maculatum', 29: 'axolotl, mu

*classification* of a single image

In [ ]:
# your file name
img_file = './drive/My Drive/public/data/images/cat_224.jpg'

input_img = load_img(img_file) # prepare an jpg image for the model
input_data = transform_img(input_img).to(device) # 3x400x225 -> 3x331x331 size may differ
input_data = input_data.unsqueeze(0)           # 3x331x331 -> 1x3x331x331

output_logits = model(input_data) # 1x1000

print("{} is [{}: {}]".format(img_file ,output_logits.argmax(),
                           imagenet_class[int(output_logits.argmax())]))

./drive/My Drive/public/data/images/cat_224.jpg is [3: tiger shark, Galeocerdo cuvieri]


classification of all the files in a directory
./drive/My Drive/public/data/images/



In [ ]:
import glob
dir_path = './drive/My Drive/public/data/images/'
img_list = glob.glob(dir_path+'*.*')

for img_file in img_list:
  input_img = load_img(img_file)
  input_data = transform_img(input_img).to(device) # -> 3x331x331
  input_data = input_data.unsqueeze(0)    # 3x331x331 -> 1x3x331x331
  
  output_logits = model(input_data) # 1x1000

  print("{} is [{}: {}]".format(img_file.split('/')[-1] ,output_logits.argmax(),
                           imagenet_class[int(output_logits.argmax())]))
  


test_0.jpg is [49: African crocodile, Nile crocodile, Crocodylus niloticus]
test_1.jpg is [282: tiger cat]
cat_224.jpg is [282: tiger cat]


# **Transfer** **Learning**


Let's see our pre-trained model

In [ ]:
print(model)
model.last_linear.in_features

NASNetALarge(
  (conv0): Sequential(
    (conv): Conv2d(3, 96, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(96, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (cell_stem_0): CellStem0(
    (conv_1x1): Sequential(
      (relu): ReLU()
      (conv): Conv2d(96, 42, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(42, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (comb_iter_0_left): BranchSeparables(
      (relu): ReLU()
      (separable_1): SeparableConv2d(
        (depthwise_conv2d): Conv2d(42, 42, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), groups=42, bias=False)
        (pointwise_conv2d): Conv2d(42, 42, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (bn_sep_1): BatchNorm2d(42, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (separable_2): SeparableConv2d(
        (depthwise_conv2d): Conv2d(42, 42, kernel_size=(5, 5), stride=(

4032

**We want to reuse pre-trained model's parameters except the 'last_linear' layer**

Let's redefine the model

In [ ]:
from pretrainedmodels.models.nasnet import NASNetALarge 
output_dim = 5

class TransferedModel(NASNetALarge) : 
  def __init__(self):
    super(TransferedModel, self).__init__()
    
    self.last_linear = nn.Linear(model.last_linear.in_features, output_dim) # 

we train only the 'last_linear' layer

In [ ]:
import torch.nn as nn

pretrained_dict = model.state_dict()

model = TransferedModel().to(device)
model_dict = model.state_dict()

## load pretrained model's parameter
# 1. filter out unnecessary keys
pretrained_dict = {k: v for k, v in pretrained_dict.items() if not ('last_linear' in k) }
# 2. overwrite entries in the existing state dict
model_dict.update(pretrained_dict)
# 3. load the new state dict
model.load_state_dict(model_dict)
# why updating the state_dict?
learning_rate = 0.0005
loss_func = nn.CrossEntropyLoss()

## optimizer without pretrained parameters
# try training also the (last-1)th layer 
for name, param in model.named_parameters() :
  if 'last_linear' in name :
    param.requires_grad = True
  else :
    param.requires_grad = False
params = filter(lambda p: p.requires_grad, model.parameters()) # only iterates through p.requires_grad == True elements
optimizer = torch.optim.Adam(params, lr=learning_rate)



*   data loader



In [ ]:
from torchvision import datasets

batch_size = 64 # try different batch_size values
data_dir = './drive/My Drive/public/'
train_dir = 'train'
valid_dir = 'valid'

train_set = datasets.ImageFolder(data_dir+train_dir, transform_img)
valid_set = datasets.ImageFolder(data_dir+valid_dir, transform_img)


train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                              shuffle=True, num_workers=4)          
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size,
                                              shuffle=True, num_workers=4)

train_size = len(train_set)
valid_size = len(valid_set)

class_names = train_set.classes

print(class_names) 
print(f'Train image size: {train_size}')
print(f'Validation image size: {valid_size}')

['ANH', 'HDH', 'Hyoam', 'NTH', 'OH']
Train image size: 4054
Validation image size: 241




*   training the new network with new dataset
 * this takes some time (less than 10 min)


In [ ]:
import os

result_dir = 'drive/My Drive/public/results/'
num_epoch = 3 # try with different epochs and find the best epoch

if not os.path.exists(result_dir):
    os.mkdir(result_dir)    
    
for i in range(num_epoch):
    model.train()
    for j, [image,label] in enumerate(train_loader):
        x = image.to(device)
        y_= label.to(device)
        
        optimizer.zero_grad()
        output = model(x)
        loss = loss_func(output,y_)
        loss.backward()
        optimizer.step()
        
        if j % 30 == 0:
            print(i,j, loss.data.cpu())
    
    model.eval()
    hits = 0
    for k, [image,label] in enumerate(valid_loader):
        x = image.to(device)
        y_= label.to(device)

        output = model(x)
        y_est = output.argmax(1)
        hits = hits + sum(y_est == y_).cpu()
    print('Hits', int(hits), 'Accuracy', float(hits/(valid_size+0.0)))    

torch.save(model, result_dir + 'teamX.model')
print('training is done by max_epochs', num_epoch)

0 0 tensor(1.6408)
0 30 tensor(0.4788)
0 60 tensor(0.2797)
Hits 232 Accuracy 0.9626556038856506
1 0 tensor(0.1959)
1 30 tensor(0.1751)
1 60 tensor(0.1145)
Hits 228 Accuracy 0.9460580945014954
2 0 tensor(0.1356)
2 30 tensor(0.1104)
2 60 tensor(0.1264)
Hits 233 Accuracy 0.9668049812316895
training is done by max_epochs 3




*   test your model



In [ ]:
test_batch_size = 10
result_dir = 'drive/My Drive/public/results/'
model_name = 'teamX.model'

model = torch.load(result_dir + model_name)
model.to(device)
model.eval()

test_dir = './drive/My Drive/public/valid2'
test_set = datasets.ImageFolder(test_dir, transform_img) #test_transform)
              
test_loader = torch.utils.data.DataLoader(test_set, batch_size=test_batch_size,
                                          shuffle=False, num_workers=4)

hits = 0
for k,[image,label] in enumerate(test_loader):
    x = image.to(device)
    y_= label.to(device)
  
    output = model(x)
    y_est = output.argmax(1)
    print('Target', label.numpy(), 'Prediction', y_est.cpu().numpy())
    hits = hits + sum(y_est == y_)
print('hits', int(hits),'accuracy', float(hits/(len(test_set)+0.0)))

Target [0 0 0 0 0 0 0 0 0 0] Prediction [0 0 0 0 0 0 0 0 0 0]
Target [0 0 0 0 0 0 0 0 0 0] Prediction [0 0 0 0 0 0 0 0 0 0]
Target [0 0 0 0 0 0 0 0 0 0] Prediction [0 0 0 0 0 0 0 0 0 0]
Target [0 0 0 0 0 0 0 0 0 0] Prediction [0 0 0 0 0 0 0 0 0 0]
Target [1 1 1 1 1 1 1 1 1 1] Prediction [1 1 1 1 1 1 1 1 1 1]
Target [1 1 1 1 1 1 1 1 1 1] Prediction [1 1 1 1 1 1 1 1 1 1]
Target [1 1 1 1 1 1 1 1 1 1] Prediction [1 1 1 1 1 1 1 1 1 1]
Target [1 1 1 1 1 1 1 1 1 1] Prediction [1 1 1 1 1 3 1 1 1 1]
Target [2 2 2 2 2 2 2 2 2 2] Prediction [2 2 2 2 2 2 2 2 2 2]
Target [3 3 3 3 3 3 3 3 3 3] Prediction [3 3 3 3 3 3 3 3 3 3]
Target [4 4 4 4 4 4 4 4 4 4] Prediction [4 4 4 4 3 4 3 3 4 3]
hits 105 accuracy 0.9545454382896423




*   classify one image



In [ ]:
from skimage import io
from torchvision import transforms

img_name = './drive/My Drive/public/test/test1.jpg'
test_img = io.imread(img_name)
test_img = transforms.ToPILImage()(test_img)
test_data = transform_img(test_img).unsqueeze(0).to(device)

output=model(test_data)

class_id = output.argmax(dim=1).cpu().numpy()[0]
print(img_name.split('/')[-1], '==>', class_id, class_names[class_id])


test1.jpg ==> 0 ANH




*   classify all images in a directory



In [ ]:
from skimage import io
import glob

img_dir = 'drive/My Drive/public/test/'
file_list = glob.glob(img_dir + '*.*')
for img_name in file_list:
  test_img = io.imread(img_name)
  test_img = transforms.ToPILImage()(test_img)
  test_data = transform_img(test_img).unsqueeze(0).to(device)
  output=model(test_data)

  class_id = output.argmax(dim=1).cpu().numpy()[0]
  print(img_name.split('/')[-1], '==>', class_id, class_names[class_id])

D08_ANH23.jpg ==> 0 ANH
D13_ANH01.jpg ==> 0 ANH
D17_Hyoam39.jpeg ==> 2 Hyoam
D18_Hyoam40.JPG ==> 2 Hyoam
D10_ANH46.jpg ==> 0 ANH
D06_ANH32.jpg ==> 0 ANH
test1.jpg ==> 0 ANH
D12_AHN36.jpg ==> 0 ANH
D10_ANH09.jpg ==> 0 ANH
D06_HDH08.jpg ==> 1 HDH
D15_HDH04.JPG ==> 1 HDH
D06_OH29.jpg ==> 4 OH
D05_NTH03.jpg ==> 3 NTH
D08_OH29.jpg ==> 4 OH
D07_HDH17.jpg ==> 1 HDH
D18_NTH30.jpg ==> 3 NTH
D11_NTH15.jpeg ==> 3 NTH
D09_Hyoam09.JPG ==> 2 Hyoam
D14_Hyoam12.jpg ==> 2 Hyoam
D15_OH43.jpg ==> 4 OH
D11_Hyoam05.JPG ==> 2 Hyoam
D04_OH18.jpeg ==> 4 OH
D01_Hyoam41.jpg ==> 2 Hyoam
D05_NTH31.jpg ==> 3 NTH
D13_NTH30.jpg ==> 3 NTH
D07_HDH49.jpg ==> 1 HDH
D16_OH34.jpg ==> 4 OH
D07_HDH45.jpg ==> 1 HDH
D14_OH12.jpg ==> 4 OH
D02_HDH50.jpg ==> 1 HDH
D15_Hyoam47.jpg ==> 2 Hyoam
D07_HDH07.jpg ==> 3 NTH
D13_ANH47.jpg ==> 1 HDH
D15_OH31.jpg ==> 4 OH
D01_OH22.jpg ==> 4 OH
D12_NTH15.jpg ==> 3 NTH
D05_NTH45.jpg ==> 3 NTH
D07_HDH30.jpg ==> 1 HDH
D09_NTH30.jpg ==> 3 NTH
D17_NTH08.jpg ==> 3 NTH
D08_Hyoam09.jpg ==> 2 Hyoam
D

the end!
----